In [ ]:
!pip3 install pandas
!pip3 install numpy

Importaciones

In [ ]:
import pandas as pd
import numpy as np

Carga de los datos y filtrado

In [ ]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")
wind_ava = wind_ava.filter(regex='^(datetime|energy|.*\.13)$')

Análisis de los datos

In [ ]:
# Número de instancias y características
print(f'Número de instancias: {wind_ava.shape[0]}')
print(f'Número de características: {wind_ava.shape[1]}')

# Variables categóricas y numéricas
categorical_vars = wind_ava.select_dtypes(include=['object']).columns
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns
print(f'Variables categóricas: {categorical_vars}')
print(f'Variables numéricas: {numerical_vars}')

# Valores faltantes
missing_values = wind_ava.isnull().sum()
print(f'Valores faltantes: {missing_values[missing_values > 0]}')

# Columnas constantes
constant_columns = [col for col in wind_ava.columns if wind_ava[col].nunique() <= 1]
print(f'Columnas constantes: {constant_columns}')

# Determinar si es un problema de regresión o clasificación
if wind_ava['energy'].nunique() > 2:
    print('Es un problema de regresión')
else:
    print('Es un problema de clasificación')